In [2]:
import lib.util
import numpy as np
from transformers import GPT2Tokenizer
import torch
import gc

[2, 30086, 6, 141, 32, 47, 38, 524, 2051]
'</s>'
'Hi'
','
' how'
' are'
' you'
' I'
' am'
' fine'


In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained("facebook/opt-350m")

tokens = tokenizer('Hi, how are you I am fine')['input_ids']

print(tokens)

for i in tokens:
    print("'"+tokenizer.decode(i)+"'")

In [2]:
####### find best silence token - which token, when inserted randomly into text, preserves prediction performance best?

from transformers import OPTForCausalLM
import random
import gc
model = OPTForCausalLM.from_pretrained("facebook/opt-350m")


print('creating dataset')
import nltk
files = nltk.corpus.gutenberg.fileids()
from nltk.corpus import gutenberg
dataset = ''

#random.shuffle(files)
#for file in files[0:10]:
#    doc = gutenberg.raw(file)[0:100000]
#    dataset+=doc
print(len(files))
dataset = gutenberg.raw(files[15]) # just use first jane austen book
print(files[15])

print('tokenizing')
tokenizer = GPT2Tokenizer.from_pretrained("facebook/opt-350m")
token_ids = tokenizer(dataset)['input_ids']
del dataset
print(len(token_ids))




SILENCE_TOKEN = 1437 #comma=6. newline character=50118. single space=1437   # run=422 : for stress test purposes
batch_size=8
num_tokens = 200
print("creatin batch", batch_size, num_tokens)
LM_input = torch.zeros([batch_size,num_tokens], dtype=torch.long)
random.seed(1)
for b in range(batch_size):

    # get random tokens from text
    start=random.randint(0,len(token_ids)-num_tokens)
    end=start+num_tokens
    LM_input[b,:] = torch.tensor(token_ids[start:end], dtype=torch.long)

    # insert silence tokens
    for t in range(num_tokens-2): # leave last two tokens for prediction
        if random.random() < 0.1: #0.059885932
            LM_input[b,t:num_tokens].roll(1)
            LM_input[b,t] = SILENCE_TOKEN
            pass

LM_input[:,0]=2

print("LM")
labels = torch.full(LM_input.shape, -100, dtype=torch.long)
labels[:,num_tokens-2] = LM_input[:,num_tokens-1]

output = model(input_ids=LM_input, labels=LM_input)

print(output.loss)

creating dataset
18
shakespeare-hamlet.txt
tokenizing
51667
creatin batch 8 200
LM
tensor(4.6932, grad_fn=<NllLossBackward>)


In [ ]:
# test gated cross attention
# make module

from lib.masked_attention import MaskedAttention
#REQUIRES:
import functools
import torch as th
from torch import nn
import lib.xf as xf
from lib.minecraft_util import store_args
from lib.tree_util import tree_map

mask_attention_module = MaskedAttention(
        input_size=128,
        memory_size=256,
        heads=2,
        timesteps=128,
        mask = "clipped_causal",
        init_scale=1,
        norm="none",
        log_scope="sa",
        use_muP_factor=False) 

# put matrix through module

import torch as th

array = th.normal(0,1,(1000,128,16,16))
print(array.mean())
array2 = array[:,0,None,None]
print(array2.shape)
print(array2.mean())



In [ ]:
# test token timings and how often silence occurs
import numpy as np
import random

result = 0
for i in range(500):
    prob_word = 0.1666666666
    d=10
    total = 0
    num_shown = 0
    skipped=0

    random_seq = []
    for i in range(2048):
        #random_seq.append(random.gauss(0.5,0.335)<prob_word)   #  np.random.uniform(0,1,2048)
        random_seq.append(random.random()<prob_word)   #  np.random.uniform(0,1,2048)
        

    for i in range(2048):

        if i%d==0 and i>0:
            token=np.max(random_seq[i-d:i])
            if (token < np.sum(random_seq[i-d:i])):
                skipped+=1
            total+=token
            num_shown += 1 #for every group checked only output 1

    result+=total/num_shown
